# Data Loading

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
test = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

def get_test_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/test/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

train['file_path'] = train['id'].apply(get_train_file_path)
test['file_path'] = test['id'].apply(get_test_file_path)

display(train.head())
display(test.head())

# Visualize

In [ ]:
import librosa
import librosa.display

for i in range(1):
    waves = np.load(test.loc[i, 'file_path']).astype(np.float32) # (3, 4096)
    melspecs = []
    for j in range(3):
        melspec = librosa.feature.melspectrogram(waves[j] / max(waves[j]),
                                                 #sr=4096, n_mels=128, fmin=20, fmax=2048)
                                                 sr=2048, n_mels=128, fmin=20, fmax=1024)
        melspec = librosa.power_to_db(melspec)
        melspec = melspec.transpose((1, 0))
        melspecs.append(melspec)
    image = np.vstack(melspecs)
    plt.imshow(image)

# Save images

In [ ]:
OUT_DIR = "test/"
os.mkdir(OUT_DIR)

In [ ]:
def save_images(file_path):
    file_name = file_path.split('/')[-1].split('.npy')[0]
    waves = np.load(file_path).astype(np.float32) # (3, 4096)
    melspecs = []
    for j in range(3):
        melspec = librosa.feature.melspectrogram(waves[j] / max(waves[j]),
                                                 sr=2048, n_mels=128, fmin=20, fmax=1024)
        melspec = librosa.power_to_db(melspec)
        melspec = melspec.transpose((1, 0))
        melspecs.append(melspec)
    image = np.vstack(melspecs)
    np.save(OUT_DIR + file_name, image)

In [ ]:
import joblib
from tqdm.auto import tqdm

_ = joblib.Parallel(n_jobs=8)(
    joblib.delayed(save_images)(file_path) for file_path in tqdm(test['file_path'].values)
)

In [ ]:
import shutil

shutil.make_archive(OUT_DIR, 'zip', OUT_DIR)
shutil.rmtree(OUT_DIR)